In [13]:
import pandas as pd

900000

In [ ]:
# Question 1: list all user with loyalty status, and all their rides by day
rides_dt = pd.read_csv("./raw_data/rides.csv", delimiter=",")
users_dt = pd.read_csv("./raw_data/users.csv", delimiter=",")

joined_df = pd.merge(rides_dt, users_dt, on=['user_id'])

joined_df['daily_date'] = pd.to_datetime(rides_dt['quote_date']).dt.date
joined_df.set_index(['user_id', 'daily_date'], inplace=True)

by_user_and_date = joined_df.groupby(['user_id', 'daily_date'])
joined_df['nb_rides'] = by_user_and_date['price_nominal'].count()
joined_df['total_price'] = by_user_and_date['price_nominal'].sum()

result = joined_df[['loyalty_status', 'loyalty_status_txt', 'nb_rides', 'total_price']]

result.head()